<a href="https://colab.research.google.com/github/pushkaraj007/GitHubExpt03_grno/blob/main/Roberta_sentiment_optimzied.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 18.0 MB/s eta 0:00:00


In [ ]:
import joblib
from scipy.special import softmax

# Load the saved RoBERTa model, tokenizer, and model name
loaded_model_dict = joblib.load('/content/drive/MyDrive/datasets/roberta_sentiment_model.pkl')
loaded_model = loaded_model_dict['model']
loaded_tokenizer = loaded_model_dict['tokenizer']
model_name = loaded_model_dict['model_name']

# Input a statement
statement = input("Enter a statement for sentiment analysis: ")

# Apply sentiment analysis using the loaded model
encoded_statement = loaded_tokenizer(statement, return_tensors='pt')
output = loaded_model(**encoded_statement)
scores = output.logits[0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'negative': scores[0],
    'neutral': scores[1],
    'positive': scores[2]
}

# Find the key with the largest value
max_key = max(scores_dict, key=scores_dict.get)

# Print the sentiment prediction
print(f"Sentiment: {max_key} (Model: {model_name})")


Enter a statement for sentiment analysis: good good doggy
Sentiment: positive (Model: cardiffnlp/twitter-roberta-base-sentiment)


In [ ]:
import joblib
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import os

# Load the saved RoBERTa model, tokenizer, and model name
loaded_model_dict = joblib.load('/content/drive/MyDrive/datasets/roberta_sentiment_model.pkl')
loaded_model = loaded_model_dict['model']
loaded_tokenizer = loaded_model_dict['tokenizer']
model_name = loaded_model_dict['model_name']

# Define a function for sentiment analysis
def analyze_sentiment(statement):
    encoded_statement = loaded_tokenizer(statement, return_tensors='pt')
    output = loaded_model(**encoded_statement)
    scores = output.logits[0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'negative': scores[0],
        'neutral': scores[1],
        'positive': scores[2]
    }
    max_key = max(scores_dict, key=scores_dict.get)
    return max_key

# Load or initialize the last processed row/index
last_processed_row = 0

# Specify the local path for the dynamic CSV file
local_csv_file_path = '/content/drive/MyDrive/datasets/water_feedback_data - water_feedback_data.csv'
# Replace with your local file path

# Check if the CSV file exists
if not os.path.isfile(local_csv_file_path):
    print("CSV file does not exist locally. Make sure it's in the specified path.")
else:
    # Read the CSV file
    df = pd.read_csv(local_csv_file_path)

    # Filter for unprocessed rows (where 'sentiment' column is empty)
    unprocessed_df = df[df['sentiment'].isna()]

    # Check if there are unprocessed rows
    if not unprocessed_df.empty:
        # Apply sentiment analysis to unprocessed rows
        for index, row in unprocessed_df.iterrows():
            statement = row['water_comment']
            sentiment = analyze_sentiment(statement)
            df.at[index, 'sentiment'] = sentiment

        # Update the last processed row/index
        last_processed_row = unprocessed_df.index.max() + 1

        # Save the updated DataFrame back to the local CSV file
        df.to_csv(local_csv_file_path, index=False)

        print("Sentiments assigned to unprocessed rows and saved to the local CSV file.")
    else:
        print("No new unprocessed rows to process.")


No new unprocessed rows to process.


In [ ]:
import joblib
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import os
from scipy.special import softmax

# Load the saved RoBERTa model, tokenizer, and model name
loaded_model_dict = joblib.load('/content/drive/MyDrive/datasets/roberta_sentiment_model.pkl')
loaded_model = loaded_model_dict['model']
loaded_tokenizer = loaded_model_dict['tokenizer']
model_name = loaded_model_dict['model_name']

# Define a function for sentiment analysis
def analyze_sentiment(statement):
    encoded_statement = loaded_tokenizer(statement, return_tensors='pt')
    output = loaded_model(**encoded_statement)
    scores = output.logits[0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'negative': scores[0],
        'neutral': scores[1],
        'positive': scores[2]
    }
    max_key = max(scores_dict, key=scores_dict.get)
    return max_key

# Load or initialize the last processed row/index
last_processed_row = 0

# Specify the local path for the dynamic CSV file
local_csv_file_path = '/content/drive/MyDrive/datasets/water_data_all_combinations - water_data_all_combinations(2).csv'
# Replace with your local file path

# Check if the CSV file exists
if not os.path.isfile(local_csv_file_path):
    print("CSV file does not exist locally. Make sure it's in the specified path.")
else:
    # Read the CSV file
    df = pd.read_csv(local_csv_file_path)

    # Check if the 'sentiment' column exists, and if not, create it
    if 'sentiment' not in df.columns:
        df['sentiment'] = None

    # Filter for unprocessed rows (where 'sentiment' column is empty)
    unprocessed_df = df[df['sentiment'].isna()]

    # Check if there are unprocessed rows
    if not unprocessed_df.empty:
        # Apply sentiment analysis to unprocessed rows
        for index, row in unprocessed_df.iterrows():
            statement = row['comments']
            sentiment = analyze_sentiment(statement)
            df.at[index, 'sentiment'] = sentiment

        # Update the last processed row/index
        last_processed_row = unprocessed_df.index.max() + 1

        # Save the updated DataFrame back to the local CSV file
        df.to_csv(local_csv_file_path, index=False)

        print("Sentiments assigned to unprocessed rows and saved to the local CSV file.")
    else:
        print("No new unprocessed rows to process.")


No new unprocessed rows to process.


In [ ]:
import pandas as pd
import plotly.express as px
from datetime import date, timedelta

# Load the data from the CSV file
data = pd.read_csv('/content/drive/MyDrive/datasets/water_data_all_combinations - water_data_all_combinations (1).csv')  # Replace with your CSV file

# Input from the user for ward, beats, and tbs
ward = input("Enter the ward: ")
beats = input("Enter the beats: ")
tbs = input("Enter the tbs: ")

# Convert the user input to datetime.date
end_date = date.today()
start_date = end_date - timedelta(days=30)  # Last 30 days

# Filter data based on user input and date range
filtered_data = data[
    (data['ward'] == ward) &
    (data['beats'] == beats) &
    (data['tbs'] == tbs) &
    (pd.to_datetime(data['date']).dt.date >= start_date) &
    (pd.to_datetime(data['date']).dt.date <= end_date)
]

# Count sentiments
sentiment_counts = filtered_data['sentiment'].value_counts().reset_index()
sentiment_counts.columns = ['Sentiment', 'Count']

# Create a Plotly pie chart
fig = px.pie(sentiment_counts, values='Count', names='Sentiment', title=f'Sentiments for Ward: {ward}, Beats: {beats}, TBS: {tbs} (Last Month)')
fig.show()


Enter the ward: w1
Enter the beats: b2
Enter the tbs: t1
